In [1]:
import os
import re
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path = os.path.dirname(os.path.abspath('Transients_MetaData.ipynb'))
total_files=list()
cleaned_files=list()
ids=list()

In [3]:
for file in os.listdir("../TESS/transients/processed_curves"):
    total_files.append(file)
    if bool(re.search(r"\blc_\d+[a-zA-Z]{1,5}_processed\b", file)):
        cleaned_files.append(file)
        ids.extend(re.findall("_(.*?)_processed", file))   

In [4]:
print(len(cleaned_files), len(total_files), len(ids))

3270 3271 3270


In [12]:
len(set(ids))

3270

In [34]:
diff = set(total_files)-set(cleaned_files)

In [35]:
diff

{'.DS_Store'}

In [36]:
data = pd.read_csv("../TESS/transients/transients.txt", sep=r'\s+', header=None)
data.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,327.160083,-43.380019,17.800,1340.34000,SN,SNIa,2018evo,ASAS-SN,1,1,1860.105,322.643
1,1,315.033417,-40.358594,17.869,1341.92600,SN,SNIa,2018exc,ATLAS,1,1,528.652,1304.728
2,1,313.474583,-38.015594,18.781,1347.90100,SN,SNII,2018fgb,ATLAS,1,1,424.192,1751.547
3,1,332.648000,-27.250161,16.700,1331.63000,SN,SNII,2018emt,ASAS-SN,1,3,642.568,1607.839
4,1,327.584333,-26.300131,19.405,1337.96500,AT,Unclassified,2018esp,ATLAS,1,3,1303.439,1157.924
5,1,333.257750,-27.671597,18.889,1337.97100,AT,Unclassified,2018esw,ATLAS,1,3,586.494,1712.718
6,1,331.012583,-22.352822,20.790,1336.98000,AT,Unclassified,2018ewp,Pan-STARRS1,1,3,521.615,711.170
7,1,323.422667,-22.692589,20.290,1337.91300,AT,Unclassified,2018ezk,Pan-STARRS1,1,3,1692.933,332.070
8,1,327.721750,-22.455419,20.810,1344.94100,AT,Unclassified,2018fkr,Pan-STARRS1,1,3,1028.213,532.398
9,1,330.474792,-22.490219,20.660,1343.97899,AT,Unclassified,2018gau,Pan-STARRS1,1,3,613.672,701.318


In [37]:
column_names = ["sector","ra","dec","magnitude at discovery","time of discovery in TESS JD","SN","classification",
                "IAU name","discovery survey","cam","ccd","column","row"]

In [38]:
data.columns = column_names
data.head()

,sector,ra,dec,magnitude at discovery,time of discovery in TESS JD,SN,classification,IAU name,discovery survey,cam,ccd,column,row
0,1,327.160083,-43.380019,17.800,1340.340,SN,SNIa,2018evo,ASAS-SN,1,1,1860.105,322.643
1,1,315.033417,-40.358594,17.869,1341.926,SN,SNIa,2018exc,ATLAS,1,1,528.652,1304.728
2,1,313.474583,-38.015594,18.781,1347.901,SN,SNII,2018fgb,ATLAS,1,1,424.192,1751.547
3,1,332.648000,-27.250161,16.700,1331.630,SN,SNII,2018emt,ASAS-SN,1,3,642.568,1607.839
4,1,327.584333,-26.300131,19.405,1337.965,AT,Unclassified,2018esp,ATLAS,1,3,1303.439,1157.924


In [39]:
classes = data['classification'].unique()

In [40]:
print(len(classes), "\n\n", classes)

26 

 ['SNIa' 'SNII' 'Unclassified' 'CV' 'SLSN-I' 'AGN' 'SNIc-BL' 'SNIbn' 'SN'
 'FRB' 'SNIc' 'SNIb/c' 'SNIIn' 'SNIb' 'SNIa-91T-like' 'Mdwarf' 'SNIIb'
 'Nova' 'SNIa-91bg-like' 'Other' 'SNIIP' 'SNI' 'Varstar' 'SNII-pec'
 'SNIa-pec' 'SNIa-SC']


In [41]:
data.shape

(4403, 13)

In [42]:
iau_name = data['IAU name'].unique()

In [43]:
print(len(iau_name), "\n\n",iau_name)

4400 

 ['2018evo' '2018exc' '2018fgb' ... '2022gn' '2022op' '2022eg']


In [51]:
data_= data[data.duplicated(subset=['IAU name'])]

In [52]:
data_

,sector,ra,dec,magnitude at discovery,time of discovery in TESS JD,SN,classification,IAU name,discovery survey,cam,ccd,column,row
1113,20,124.242708,50.276275,19.441,1845.140,AT,Unclassified,2019yxa,ATLAS,2,4,1305.239,2.369
2821,32,76.316517,-7.285450,16.600,2176.040,AT,Varstar,2020aavs,ASAS-SN,2,3,1689.547,13.705
2986,34,120.373587,-10.169608,20.930,2229.942,AT,Unclassified,2021etl,Pan-STARRS1,2,4,312.216,13.785


In [48]:
x=data_['IAU name'].to_list()
x

['2019yxa', '2020aavs', '2021etl']

In [18]:
data[data['IAU name']=='2019yxa']

,sector,ra,dec,magnitude at discovery,time of discovery in TESS JD,SN,classification,IAU name,discovery survey,cam,ccd,column,row
1022,20,124.242708,50.276275,19.441,1845.14,AT,Unclassified,2019yxa,ATLAS,1,1,842.693,3.582
1113,20,124.242708,50.276275,19.441,1845.14,AT,Unclassified,2019yxa,ATLAS,2,4,1305.239,2.369


In [19]:
data[data['IAU name']=='2020aavs']

,sector,ra,dec,magnitude at discovery,time of discovery in TESS JD,SN,classification,IAU name,discovery survey,cam,ccd,column,row
2759,32,76.316517,-7.28545,16.6,2176.04,AT,Varstar,2020aavs,ASAS-SN,1,2,456.736,0.609
2821,32,76.316517,-7.28545,16.6,2176.04,AT,Varstar,2020aavs,ASAS-SN,2,3,1689.547,13.705


In [20]:
data[data['IAU name']=='2021etl']

,sector,ra,dec,magnitude at discovery,time of discovery in TESS JD,SN,classification,IAU name,discovery survey,cam,ccd,column,row
2925,34,120.373587,-10.169608,20.93,2229.942,AT,Unclassified,2021etl,Pan-STARRS1,1,1,1834.796,7.544
2986,34,120.373587,-10.169608,20.93,2229.942,AT,Unclassified,2021etl,Pan-STARRS1,2,4,312.216,13.785


In [21]:
diff_ids =set(iau_name)-set(ids)

In [22]:
print(len(diff_ids))


1130


In [23]:
sn_ = data['SN'].unique()

In [24]:
print(sn_)

['SN' 'AT']


In [25]:
data.drop_duplicates(subset=['IAU name'], keep='last')

,sector,ra,dec,magnitude at discovery,time of discovery in TESS JD,SN,classification,IAU name,discovery survey,cam,ccd,column,row
0,1,327.160083,-43.380019,17.800,1340.340,SN,SNIa,2018evo,ASAS-SN,1,1,1860.105,322.643
1,1,315.033417,-40.358594,17.869,1341.926,SN,SNIa,2018exc,ATLAS,1,1,528.652,1304.728
2,1,313.474583,-38.015594,18.781,1347.901,SN,SNII,2018fgb,ATLAS,1,1,424.192,1751.547
3,1,332.648000,-27.250161,16.700,1331.630,SN,SNII,2018emt,ASAS-SN,1,3,642.568,1607.839
4,1,327.584333,-26.300131,19.405,1337.965,AT,Unclassified,2018esp,ATLAS,1,3,1303.439,1157.924
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4398,47,167.461817,75.334383,20.670,2601.001,AT,Unclassified,2022aji,Pan-STARRS1,3,2,521.417,1084.865
4399,47,210.216792,78.085778,19.680,2601.143,AT,Unclassified,2022aha,Pan-STARRS1,3,3,1679.425,1415.721
4400,47,300.459933,85.849581,18.075,2588.733,AT,Unclassified,2022gn,ATLAS,3,4,1667.608,986.600
4401,47,256.141083,76.155150,18.470,2589.662,AT,Unclassified,2022op,GaiaAlerts,4,1,1990.840,279.007


In [26]:
data.reset_index()

,index,sector,ra,dec,magnitude at discovery,time of discovery in TESS JD,SN,classification,IAU name,discovery survey,cam,ccd,column,row
0,0,1,327.160083,-43.380019,17.800,1340.340,SN,SNIa,2018evo,ASAS-SN,1,1,1860.105,322.643
1,1,1,315.033417,-40.358594,17.869,1341.926,SN,SNIa,2018exc,ATLAS,1,1,528.652,1304.728
2,2,1,313.474583,-38.015594,18.781,1347.901,SN,SNII,2018fgb,ATLAS,1,1,424.192,1751.547
3,3,1,332.648000,-27.250161,16.700,1331.630,SN,SNII,2018emt,ASAS-SN,1,3,642.568,1607.839
4,4,1,327.584333,-26.300131,19.405,1337.965,AT,Unclassified,2018esp,ATLAS,1,3,1303.439,1157.924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4398,4398,47,167.461817,75.334383,20.670,2601.001,AT,Unclassified,2022aji,Pan-STARRS1,3,2,521.417,1084.865
4399,4399,47,210.216792,78.085778,19.680,2601.143,AT,Unclassified,2022aha,Pan-STARRS1,3,3,1679.425,1415.721
4400,4400,47,300.459933,85.849581,18.075,2588.733,AT,Unclassified,2022gn,ATLAS,3,4,1667.608,986.600
4401,4401,47,256.141083,76.155150,18.470,2589.662,AT,Unclassified,2022op,GaiaAlerts,4,1,1990.840,279.007


In [28]:
data.to_csv(path_or_buf="../TESS/transients/transients.csv", header=True, index=False)